In [3]:
# IB-MULTI-STRATEGY-ATS/broker/functions.py
import numpy as np
import pandas as pd
import yfinance as yf
from ib_insync import *
# from gui.log import add_log
import datetime

util.startLoop()  # Needed in script mode
ib = IB()
try:
    ib.connect('127.0.0.1', 7497, clientId=5)
except ConnectionError:
    print('Connection failed. Start TWS or IB Gateway and try again!')
except:
    ib.connect('127.0.0.1', 7497, clientId=1)


In [4]:
future_symbol = "ES"
index_symbol = "SPX"

In [8]:
today = datetime.datetime.now()

# Generate contract months for the next 9 maturities
monthly_expirations = [
    f"{today.year + (today.month + i - 1) // 12}{(today.month + i - 1) % 12 + 1:02}" 
    for i in range(12)
]

# Create Future objects for each contract month
contracts = [Future(future_symbol, lastTradeDateOrContractMonth=exp) for exp in monthly_expirations]
qualified_contracts = ib.qualifyContracts(*contracts)
ib.sleep(1)

# Set market data type to delayed frozen data
ib.reqMarketDataType(3)

idx = Index(index_symbol)
ib.qualifyContracts(idx)
idx_details = ib.reqMktData(idx)
ib.sleep(1)
spot = idx_details.last

futures_data = {
    'Contract': ["Index"],
    'LastPrice': [spot],
    'DTE': [0],
    'AnnualizedYield': [None]
}

if not qualified_contracts:
    contracts = [Future(future_symbol, exchange="CME" ,lastTradeDateOrContractMonth=exp) for exp in monthly_expirations]
    qualified_contracts = ib.qualifyContracts(*contracts)

Error 200, reqId 29: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202401')
Error 200, reqId 30: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202402')
Error 200, reqId 32: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202404')
Error 200, reqId 33: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202405')
Error 200, reqId 35: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202407')
Error 200, reqId 36: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202408')
Error 200, reqId 38: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractM

Peer closed connection.
Peer closed connection.


In [39]:
qualified_contracts
contract = qualified_contracts[0]

In [6]:
market_data = ib.reqMktData(contract)
ib.sleep(1)  # Wait for the data to be fetched
last_price = market_data.last if market_data.last > 0 else market_data.bid

expiration_date = datetime.datetime.strptime(contract.lastTradeDateOrContractMonth, '%Y%m%d')
dte = (expiration_date - today).days
annualized_yield = None

if dte != 0:
    annualized_yield = ((last_price / spot) ** (365 / dte) - 1) if last_price <= spot else ((spot / last_price) ** (365 / dte) - 1)

futures_data['Contract'].append(contract.localSymbol)
futures_data['LastPrice'].append(last_price)
futures_data['DTE'].append(dte)
futures_data['AnnualizedYield'].append(annualized_yield)

# Convert the dictionary to DataFrame
term_structure = pd.DataFrame(futures_data)

NameError: name 'contract' is not defined

In [7]:
term_structure

NameError: name 'term_structure' is not defined